# The New-Keynesian Model

As opposed to the Real Business Cycle model by Prescott, which hypothesizes that the main driver of changes in macroeconomic quantities over the Real Business Cycle is caused by changes in the "A" in the production function, the New-Keynsian model instead places the ownership at the feet of demand, supply, and monetary policy shocks.

Thus, the goal of this experiment will be to analyze the different impulse responses to output and inflation in the case of different forms of shocks: demand, supply, and monetary policy shocks.

In [18]:
#Install linearsolve to be able to approximate solutions to FOCs
!pip install linearsolve

In [19]:
#Import packages
import numpy as np #Numpy for array operations
import pandas as pd #Pandas for dataframes and data management
import linearsolve as ls #Linearsolve for FOC computation
plt.style.use('classic')
plt.rcParams['figure.facecolor'] = 'white'

## The Equilibrium Conditions in the New-Keynesian Model

We will utilize the following set of equations to define our New-Keynesian model

\begin{align}
y_t & = E_t y_{t+1}  - \left( r_{t} - \bar{r}\right) + g_t\\
i_{t} & = r_{t} + E_t \pi_{t+1}\\
i_{t} & = \bar{r} + \pi^T + \phi_{\pi}\big(\pi_t - \pi^T\big) + \phi_{y}\big(y_t - \bar{y}\big) + v_t\\
\pi_t -\pi^T & = \beta \left( E_t\pi_{t+1} - \pi^T\right)  + \kappa (y_t -\bar{y})+ u_t,
\end{align}

where: $y_t$ is the log transform of output, $r_t$ is the real interest rate, $i_t$ is the nominal interest rate, $\pi_t$ is the rate of inflation between periods $t-1$ and $t$, $\bar{r}$ is the long-run average real interest rate or the *natural rate of interest*, $\beta$ is the household's subjective discount factor, and $\pi^T$ is the central bank's inflation target. The coeffieints $\phi_{\pi}$ and $\phi_{y}$ can be viewed as the "hatred" coefficients, with a larger coefficient indicating that the central bank really will respond more heavily to, with the first being inflation and the second being output. I.e., the higher $\phi_{\pi}$ is, the more the central bank hates inflation, and would thus respond more harshly to keep in check.

We represent g as the shock to aggregate demand, u as the shock for inflation, and v as the shock to monetary policy.

\begin{align}
g_{t+1} & = \rho_g g_{t} + \epsilon^g_{t+1}\\
u_{t+1} & = \rho_u u_{t} + \epsilon^u_{t+1}\\
v_{t+1} & = \rho_v v_{t} + \epsilon^v_{t+1}.
\end{align}

These are the parameters we will be utilizing in our model:

| $$\bar{y}$$ | $$\beta$$ | $$\bar{r}$$    | $$\kappa$$ | $$\pi^T$$ | $$\phi_{\pi}$$ | $$\phi_y$$ | $$\rho_g$$ | $$\rho_u$$ | $$\rho_v$$ | 
|-------------|-----------|----------------|------------|-----------|----------------|------------|------------|------------|------------|
| 0           | 0.995     | $$-\log\beta$$ | 0.25       | 0.02/4    | 1.5            | 0.5/4      | 0.5        | 0.5        | 0.5        |

## Solution

In [22]:
#Create a pandas series to store our parameters in the model
parameters = pd.Series({
    'y_bar': 0,
    'beta': 0.995,
    'r_bar': -np.log(0.995),
    'kappa': 0.25,
    'pi_T': 0.02/4,
    'phi_pi': 1.5,
    'phi_y': 0.5/4,
    'rho_g': 0.5,
    'rho_u': 0.5,
    'rho_v': 0.5
})

#Store exogenous state variables in a list
exo_states = ['g','u','v']

#Store co-state variables in a list
costates = ['y','pi','i','r']

#Create function to compute the solutions to our FOCs
def equations(variables_forward, variables_current, parameters):
    
    p = parameters #parameters
    cur = variables_current #variable values at time t
    fwd = variables_forward #variable values at time t+1

    #Equilibrium Conditions, cur. is the time t value of the variable, fwd. is the time t+1 value of the variable
    is_equation = fwd.y - (cur.r - p.r_bar) + cur.g - cur.y
    fisher_equation = cur.r + fwd.pi - cur.i 
    monetary_policy = p.r_bar + p.pi_T + p.phi_pi*(cur.pi - p.pi_T) + p.phi_y*cur.y + cur.v - cur.i
    phillips_curve = p.beta*(fwd.pi - p.pi_T) + p.kappa*cur.y + cur.u - (cur.pi - p.pi_T)
    demand_process = p.rho_g*cur.g - fwd.g
    monetary_policy_process = p.rho_v*cur.v - fwd.v
    inflation_process = p.rho_u*cur.u - fwd.u

    #Place everything into an array
    return np.array([is_equation, fisher_equation, monetary_policy, phillips_curve, demand_process, monetary_policy_process, inflation_process])

#Initiate model
nk_model = ls.model(equations=equations, exo_states=exo_states, costates=costates, parameters=parameters)

#Compute steady state
nk_model.compute_ss()


In [23]:
#Compute our linear approximation to the problem
nk_model.approximate_and_solve()

In [24]:
#Compute the impulse responses to our variables with a shock
nk_model.impulse(T=21,t0=5,shocks=[0.01,0.01/4,0.01/4],normalize=False)

### Demand Shock

Here we will explore the impacts of a shock to aggregate demand on the impulse responses of inflation and output in the New-Keynesian model

In [26]:
#We will utilize bokeh plotting for nicer looking graphs
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook

output_notebook()  #This will allow for dynamic viewing of graphs & interactability

Loading BokehJS ...

In [28]:
#Create our data for our graphs: variables and scale
data = {
    'time': nk_model.irs['e_g']['i'].index, 
    'nominal_interest': nk_model.irs['e_g']['i'] * 400,
    'real_interest': nk_model.irs['e_g']['r'] * 400,
    'output': nk_model.irs['e_g']['y'] * 100,
    'inflation': nk_model.irs['e_g']['pi'] * 400
}

#Create our axes settings: Graph title, variable name, y axis limits, scalar, and color of graph
settings = [
    ('Nominal Interest (annualized)', 'nominal_interest', [-0.5, 3.5], 400, 'red'),
    ('Real Interest (annualized)', 'real_interest', [-0.5, 3.5], 400, 'blue'),
    ('Output', 'output', [-0.5, 2], 100, 'green'),
    ('Inflation (annualized)', 'inflation', [-0.5, 2], 400, 'orange')
]

#Initialize list to store the plots in
plots = []

#Initiate for loop to go through the graphs: loop through title of graph, the y limits of the y axis, the scalar, and the color of graph
for title, var, ylim, scale, color in settings:
    p = figure(title=title, x_axis_label='Time', y_axis_label='% dev from steady state', height=400, width=600)
    p.line(data['time'], data[var], line_width=4, color=color, alpha=0.75)
    
    #Activate the y limits desired for graph visulization
    p.y_range.start = ylim[0]
    p.y_range.end = ylim[1]
    
    #Center title for look factor
    p.title.align = 'center'  #Title is now centered
    plots.append(p) #Append the plot to the list to graph all together

#Make spacing of graphs wider to allow for better visualization
grid = gridplot([[plots[0], plots[1]], [plots[2], plots[3]]], sizing_mode='scale_width')

#Show the plot
show(grid)


## Demand Shock Results & Conclusions

What we are able to notice in the case of a demand shock, is that there is a growth in output. However, we see that there is a larger inflationary increase compared to steady state for inflation versus output. Thus, an aggregate demand shock may put upwards pressure on inflation, which creates a trade-off situation.

### Inflation Shock

We will now examine the changes in the impulse responses to real interest rates, output, and inflation in response to what we typically think of as a "supply shock", where there is some kind of new constraint placed on production such as the Oil shocks of the 1970s.

In [31]:
#Create our data for our graphs: variables and scale
data = {
    'time': nk_model.irs['e_u']['i'].index,  
    'nominal_interest': nk_model.irs['e_u']['i'] * 400,
    'real_interest': nk_model.irs['e_u']['r'] * 400,
    'output': nk_model.irs['e_u']['y'] * 100,
    'inflation': nk_model.irs['e_u']['pi'] * 400
}

#Create our axes settings: Graph title, variable name, y axis limits, scalar, and color of graph
settings = [
    ('Nominal Interest (annualized)', 'nominal_interest', 'red'),
    ('Real Interest (annualized)', 'real_interest', 'blue'),
    ('Output', 'output', 'green'),
    ('Inflation (annualized)', 'inflation', 'orange')
]

#Initialize list to store the plots in
plots = []

#Initiate for loop to go through the graphs: loop through title of graph, the y limits of the y axis, the scalar, and the color of graph
for title, var, color in settings:
    p = figure(title=title, x_axis_label='Time', y_axis_label='% dev from steady state', height=400, width=600)
    p.line(data['time'], data[var], line_width=4, color=color, alpha=0.75)
    
    #Center title for look factor
    p.title.align = 'center'  #Title is now centered
    plots.append(p) #Append the plot onto the list

#Make spacing of graphs wider to allow for better visualization
grid = gridplot([[plots[0], plots[1]], [plots[2], plots[3]]], sizing_mode='scale_width')

#Show the plot
show(grid)


## Inflation Shock Results & Conclusions

What we see is that when a supply side shock occurs, we have a rise in the real interest rate. We then have a drop in the output compared to the steady state, and a rise in inflation. The rise in inflation causes a rise in the real interest rate due to inflation expectations and the central bank trying to fight to bring the inflation down. However, this also will cause a drop in the output as well.

### Monetary Policy Shock

Here will we examine what happens to the impulse responses to inflation and output when we implement a monetary policy shock

In [34]:
#Create our data for our graphs: variables and scale
data = {
    'time': nk_model.irs['e_v']['i'].index,  # assuming time is the index of the data
    'nominal_interest': nk_model.irs['e_v']['i'] * 400,
    'real_interest': nk_model.irs['e_v']['r'] * 400,
    'output': nk_model.irs['e_v']['y'] * 100,
    'inflation': nk_model.irs['e_v']['pi'] * 400
}

#Create our axes settings: Graph title, variable name, y axis limits, scalar, and color of graph
settings = [
    ('Nominal Interest (annualized)', 'nominal_interest', 'red'),
    ('Real Interest (annualized)', 'real_interest', 'blue'),
    ('Output', 'output', 'green'),
    ('Inflation (annualized)', 'inflation', 'orange')
]

#Initialize list to store the plots in
plots = []

#Initiate for loop to go through the graphs: loop through title of graph, the y limits of the y axis, the scalar, and the color of graph
for title, var, color in settings:
    p = figure(title=title, x_axis_label='Time', y_axis_label='% dev from steady state', height=400, width=600)
    p.line(data['time'], data[var], line_width=4, color=color, alpha=0.75)
    
    #Center title for look factor
    p.title.align = 'center'  #Title is now centered
    plots.append(p) #Append plot onto list

#Make spacing of graphs wider to allow for better visualization
grid = gridplot([[plots[0], plots[1]], [plots[2], plots[3]]], sizing_mode='scale_width')

#Show the plot
show(grid)


## Monetary Policy Shock Results & Conclusions:

We see that when there is a monetary policy shock, we have both the difference between output potential and output going further negative, but inflation also going down. Thus, there's a trade-off here in that when the central bank implements a monetary policy shock, we can expect there to be a drop off in inflation which may be good depending on circumstances, but also that there's a drop in output. 